<a href="https://colab.research.google.com/github/Amaljayaranga/DavisDataset/blob/master/DavisDatasetNew.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from torch.utils.data import Dataset
import numpy as np
import json
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
import torch
import torch.nn as nn
import cv2

class DavisDataset(Dataset):

    def __init__(self, memmap, json_data, need_classes):

        ##cropping objects from memaps and add to idxes
        self.memmap = memmap
        self.json_data = json_data
        self.idxes = {}
        self.need_classes = need_classes

        imgs = self.json_data['imgs']
        for img_index, img in enumerate(imgs):
            idx = int(img['index'])
            classes = img['objects']
            classes_ = []
            objects_ = []
            class_objects = []
            for class_idx, class_ in enumerate(classes):
               if class_['class']  in self.need_classes:
                  single_class = class_['class']
                  x = class_['x']
                  y = class_['y']
                  width = class_['width']
                  height = class_['height']

                  image = self.memmap[img_index]
                  
                  crop_img = image[y:y + height, x:x + width]
                  #img_resized = cv2.resize(crop_img, (200,200), interpolation = cv2.INTER_AREA)
                  classes_.append(single_class)
                  #objects_.append(self.image_to_tensor(img_resized))
                  objects_.append(crop_img)

            class_objects.append(classes_)
            class_objects.append(objects_)
            self.idxes[idx] = class_objects

    def image_to_tensor(self,image, mean=0, std=1.):
        image = image.astype(np.float32)
        image = (image - mean) / std
        image = image.transpose((2, 0, 1))
        tensor = torch.from_numpy(image)
        return tensor

    def __len__(self):
        return len(self.memmap)

    def __getitem__(self, index):
        classes, images = self.idxes.get(index)
        return [classes, images]



#loading memmap and json for same context
folder_path='./drive/My Drive/Thesis_2020/'
complete_shape=(60,480,854,3)
complete_memmap = folder_path+'people-horse.mmap'
newfp = np.memmap(complete_memmap, dtype='uint8', mode='r', shape=complete_shape)
#loading json
with open(folder_path+'people-horse-json.txt') as json_file:
    data_json = json.load(json_file)




def show_objects(classes,images):
  img_np = []
  for i in range(len(classes)):
    img_np_i = np.asarray(images[i])
    img_np.append(img_np_i)

  f, axarr = plt.subplots(1, len(classes))
  for i in range(len(classes)):
    axarr[i].imshow(img_np[i], interpolation='nearest')
  plt.show()

need_classes = ['person','horse']
same_conext_ds = DavisDataset(newfp, data_json, need_classes)


def show_images(same_conext_ds):
  for i in range(60):
    classes, images  = same_conext_ds.__getitem__(i)
    show_objects(classes,images)

show_images(same_conext_ds)
